In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [2]:
from replay_trajectory_classification.core import _causal_classify

_causal_classify?

Signature:       _causal_classify(initial_conditions, continuous_state_transition, discrete_state_transition, likelihood)
Call signature:  _causal_classify(*args, **kwargs)
Type:            CPUDispatcher
String form:     CPUDispatcher(<function _causal_classify at 0x1144c5b70>)
File:            ~/Documents/GitHub/replay_trajectory_classification/replay_trajectory_classification/core.py
Docstring:      
Adaptive filter to iteratively calculate the posterior probability
of a state variable using past information.

Parameters
----------
initial_conditions : ndarray, shape (n_states, n_bins, 1)
continuous_state_transition : ndarray, shape (n_states, n_states,
                                              n_bins, n_bins)
discrete_state_transition : ndarray, shape (n_states, n_states)
likelihood : ndarray, shape (n_time, n_states, n_bins, 1)

Returns
-------
causal_posterior : ndarray, shape (n_time, n_states, n_bins, 1)
Class docstring:
Implementation of user-facing dispatcher objects (i.e.

In [13]:
import numpy as np

n_states, n_bins, n_time = 2, 5, 15

initial_conditions = np.ones((n_states, n_bins, 1))

continuous_state_transition = np.ones((n_states, n_states, n_bins, n_bins))
discrete_state_transition = np.identity(n_states)
likelihood = np.ones((n_time, n_states, n_bins, 1))

posterior = _causal_classify(initial_conditions, continuous_state_transition, discrete_state_transition, likelihood)
print(posterior.shape)
posterior[0]

(15, 2, 5, 1)


array([[[0.1],
        [0.1],
        [0.1],
        [0.1],
        [0.1]],

       [[0.1],
        [0.1],
        [0.1],
        [0.1],
        [0.1]]])

In [36]:
def _log_causal_classify(initial_conditions, continuous_state_transition, discrete_state_transition, likelihood):
    n_time, n_states, n_bins, _ = likelihood.shape
    posterior = np.zeros_like(likelihood)
    
    # initialize model
    for state_ind in range(n_states):
        for bin_ind in range(n_bins):
            posterior[0, state_ind, bin_ind] = initial_conditions[state_ind, bin_ind] + likelihood[0, state_ind, bin_ind]
        
    #
    
    for time_ind in range(1, n_time):
        for cur_state_ind in range(n_states):
            for prev_state_ind in range(n_states):
                for bin_ind in range(n_bins):
                    posterior[time_ind, state_ind, bin_ind] = (
                        discrete_state_transition[prev_state_ind, cur_state_ind]
                    )
    
    return posterior


log_posterior = _log_causal_classify(
    np.log(initial_conditions),
    np.log(continuous_state_transition),
    np.log(discrete_state_transition),
    np.log(likelihood))

print(log_posterior.shape)
np.exp(log_posterior[0])

(15, 2, 5, 1)


/Users/edeno/miniconda3/envs/replay_trajectory_classification/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log


array([[[1.],
        [1.],
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        [1.],
        [1.]]])

In [37]:
np.logaddexp?

Call signature:  np.logaddexp(*args, **kwargs)
Type:            ufunc
String form:     <ufunc 'logaddexp'>
File:            ~/miniconda3/envs/replay_trajectory_classification/lib/python3.6/site-packages/numpy/__init__.py
Docstring:      
logaddexp(x1, x2, /, out=None, *, where=True, casting='same_kind', order='K', dtype=None, subok=True[, signature, extobj])

Logarithm of the sum of exponentiations of the inputs.

Calculates ``log(exp(x1) + exp(x2))``. This function is useful in
statistics where the calculated probabilities of events may be so small
as to exceed the range of normal floating point numbers.  In such cases
the logarithm of the calculated probability is stored. This function
allows adding probabilities stored in such a fashion.

Parameters
----------
x1, x2 : array_like
    Input values. If ``x1.shape != x2.shape``, they must be broadcastable to a common shape (which becomes the shape of the output).
out : ndarray, None, or tuple of ndarray and None, optional
    A locatio